In [0]:
# Kafka connection configuration
bootstrap_servers = "pkc-56d1g.eastus.azure.confluent.cloud:9092"
api_key = "46E3IOCDMWEUU2CV"
api_secret = "ZjVR54y34uCeO8a5lVxgqwa6QCv6/YCCazz3vSEdFysdbYmenaPWtNBORR9wkF2w"
topic = "transactions-topic"

In [0]:
# Read Kafka stream into Spark
kafka_df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", bootstrap_servers)
    .option("subscribe", topic)
    .option("kafka.security.protocol", "SASL_SSL")
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("kafka.sasl.jaas.config", 
        f'org.apache.kafka.common.security.plain.PlainLoginModule required username="{api_key}" password="{api_secret}";')
    .option("startingOffsets", "latest")
    .load()
)

In [0]:
# Decode the Kafka value from binary to string
decoded_df = kafka_df.selectExpr("CAST(value AS STRING) AS json_string")

from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType

In [0]:
# Define schema for your JSON structure
schema = StructType() \
    .add("event_id", StringType()) \
    .add("region", StringType()) \
    .add("amount", DoubleType()) \
    .add("event_time", TimestampType())

In [0]:
# Parse the json_string column into structured columns
parsed_df = decoded_df.select(
    from_json(col("json_string"), schema).alias("data")
).select("data.*")

from pyspark.sql.functions import col
from pyspark.sql.streaming import DataStreamWriter

In [0]:
# Deduplicate on event_id with watermarking to manage memory
deduped_df = (
    parsed_df
    .withWatermark("event_time", "10 minutes")
    .dropDuplicates(["event_id"])
)

# Display deduplicated stream
display(deduped_df)



event_id,region,amount,event_time
TX001,US,10000.0,2025-06-20T16:00:00.000+0000
TX002,EU,7550.75,2025-06-20T18:45:00.000+0000
